In [1]:
import pandas as pd

In [2]:
data_raw = pd.read_csv('Data/creditcard.csv')

In [4]:
data_raw.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [1]:
# Iniciamos pyspark y arrancamos SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TFM").getOrCreate()

In [82]:
#Importamos la librerias
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler 
from pyspark.sql.functions import *
from sklearn.metrics import classification_report, confusion_matrix
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

In [3]:
# Importamos los datos
df = spark.read.csv("data/creditcard.csv", header=True, inferSchema = True)

In [13]:
# Realizamos un subsampling de los datos
df_label_1 = df.where(col("Class") == 1)
df_label_0 = df.where(col("Class") == 0).sample(False, df_label_1.count()/df.count(),123)
df_final = df_label_1.union(df_label_0).orderBy(rand())

In [32]:
# Agrupamos las columnas que usaremos para entrenar nuestro modelo
feature_columns = [col for col in df_final.columns if col.startswith("V")]
vectorizer = VectorAssembler(inputCols = feature_columns, outputCol="features")

In [36]:
# Hacemos la particiones de nuestros dataFrames en Train y Test
df_train, df_test = df_final.randomSplit([0.8, 0.2], seed = 123)

#### RAMDOMFORESTCLASSIFIER

In [37]:
from pyspark.ml.classification import RandomForestClassifier

In [47]:
# Declaramos nuestro algoritmo 
est = RandomForestClassifier()
est.setFeatureSubsetStrategy("auto")
est.setLabelCol("Class")
est.setSeed(42)

RandomForestClassifier_2da7f86fd666

In [48]:
# Conectamos mediante una tuberia nuestro vectorizer y nuestro algoritmo
pipeline = Pipeline()
pipeline.setStages([vectorizer, est])
# Entrenamos el algoritmo
model = pipeline.fit(df_train)

In [49]:
# Realizamos las predicciones
df_test_pred = model.transform(df_test)

In [50]:
# Creamos la Matrix de cunfusion 
y_true = df_test_pred.select(['Class']).collect()
y_pred = df_test_pred.select(['prediction']).collect()
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.98      0.95       101
           1       0.98      0.91      0.94        90

    accuracy                           0.95       191
   macro avg       0.95      0.95      0.95       191
weighted avg       0.95      0.95      0.95       191



In [59]:
# Comenzamos a tunear el CrossValidator

paramGrid = ParamGridBuilder()\
            .addGrid(est.maxBins,[10,50,100])\
            .addGrid(est.impurity,["entropy","gini"])\
            .addGrid(est.maxDepth,[4,6,8])\
            .build()

evaluator = BinaryClassificationEvaluator()\
            .setLabelCol("Class")\
            .setMetricName("areaUnderROC")


crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)

In [60]:
# Entrenamos y realizamos las predicciones
cvModel = crossval.fit(df_train)
df_test_pred = cvModel.transform(df_test)

In [64]:
# Introducimos nuestras predicciones en la matrix de confusion y observamos una mejora de nuestras metricas. 
y_true = df_test_pred.select(['Class']).collect()
y_pred = df_test_pred.select(['prediction']).collect()
print(classification_report(y_true, y_pred))

In [65]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       101
           1       1.00      0.91      0.95        90

    accuracy                           0.96       191
   macro avg       0.96      0.96      0.96       191
weighted avg       0.96      0.96      0.96       191



In [80]:
# Guardamos el Modelo para exportarlo a nuestro archivo de Spark Streaming. 
model = cvModel.bestModel
model.write().overwrite().save("credit-model3")

TypeError: 'PipelineModel' object is not callable